## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [33]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [34]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [35]:
import requests
import pandas as pd
from slugify import slugify
from datetime import datetime
import json

## Download

Retrieve the data

In [36]:
POWER_BI_QUERY_URL = (
    "https://wabi-us-gov-iowa-api.analysis.usgovcloudapi.net/public/reports/querydata"
)
REQUEST_PARAMS = (("synchronous", "true"), ("Connection", "keep-alive"))
# REQUEST_BODY = '{"version":"1.0.0","queries":[{"Query":{"Commands":[{"SemanticQueryDataShapeCommand":{"Query":{"Version":2,"From":[{"Name":"t1","Entity":"tab2_cities","Type":0}],"Select":[{"Column":{"Expression":{"SourceRef":{"Source":"t1"}},"Property":"cities"},"Name":"tab2_cities.cities"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"t1"}},"Property":"# Known Cases"}},"Function":0},"Name":"Sum(tab2_cities.# Known Cases)"},{"Column":{"Expression":{"SourceRef":{"Source":"t1"}},"Property":"% Known Cases"},"Name":"Sum(tab2_cities.% Known Cases)"},{"Column":{"Expression":{"SourceRef":{"Source":"t1"}},"Property":"% County Population"},"Name":"tab2_cities.% County Population"}],"OrderBy":[{"Direction":1,"Expression":{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"t1"}},"Property":"# Known Cases"}},"Function":0}}}]},"Binding":{"Primary":{"Groupings":[{"Projections":[0,1,2,3]}]},"DataReduction":{"DataVolume":3,"Primary":{"Window":{"Count":500}}},"Version":1},"ExecutionMetricsKind":1}}]},"CacheKey":"{\\"Commands\\":[{\\"SemanticQueryDataShapeCommand\\":{\\"Query\\":{\\"Version\\":2,\\"From\\":[{\\"Name\\":\\"t1\\",\\"Entity\\":\\"tab2_cities\\",\\"Type\\":0}],\\"Select\\":[{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"t1\\"}},\\"Property\\":\\"cities\\"},\\"Name\\":\\"tab2_cities.cities\\"},{\\"Aggregation\\":{\\"Expression\\":{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"t1\\"}},\\"Property\\":\\"# Known Cases\\"}},\\"Function\\":0},\\"Name\\":\\"Sum(tab2_cities.# Known Cases)\\"},{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"t1\\"}},\\"Property\\":\\"% Known Cases\\"},\\"Name\\":\\"Sum(tab2_cities.% Known Cases)\\"},{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"t1\\"}},\\"Property\\":\\"% County Population\\"},\\"Name\\":\\"tab2_cities.% County Population\\"}],\\"OrderBy\\":[{\\"Direction\\":1,\\"Expression\\":{\\"Aggregation\\":{\\"Expression\\":{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"t1\\"}},\\"Property\\":\\"# Known Cases\\"}},\\"Function\\":0}}}]},\\"Binding\\":{\\"Primary\\":{\\"Groupings\\":[{\\"Projections\\":[0,1,2,3]}]},\\"DataReduction\\":{\\"DataVolume\\":3,\\"Primary\\":{\\"Window\\":{\\"Count\\":500}}},\\"Version\\":1},\\"ExecutionMetricsKind\\":1}}]}","QueryId":"","ApplicationContext":{"DatasetId":"da83725f-fd62-480a-9a33-a73b06aa6bb1","Sources":[{"ReportId":"5663b2c5-89c2-4903-bb89-0086bbd23b8a","VisualId":"4dbc0385a9d366c66bd6"}]}}],"cancelQueries":[],"modelId":350069}'

In [37]:
REQUEST_BODY = '{"version":"1.0.0","queries":[{"Query":{"Commands":[{"SemanticQueryDataShapeCommand":{"Query":{"Version":2,"From":[{"Name":"t","Entity":"tab2_zip","Type":0}],"Select":[{"Column":{"Expression":{"SourceRef":{"Source":"t"}},"Property":"zips"},"Name":"tab2_zip.zips"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"t"}},"Property":"# Known Cases"}},"Function":0},"Name":"Sum(tab2_zip.# Known Cases)"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"t"}},"Property":"% Known Cases"}},"Function":0},"Name":"Sum(tab2_zip.% Known Cases)"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"t"}},"Property":"% County Population"}},"Function":0},"Name":"Sum(tab2_zip.% County Population)"}]},"Binding":{"Primary":{"Groupings":[{"Projections":[0,1,2,3]}]},"DataReduction":{"DataVolume":3,"Primary":{"Top":{"Count":30000}}},"Version":1,"Highlights":[{"Version":2,"From":[{"Name":"t","Entity":"tab2_zip","Type":0}],"Where":[{"Condition":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"t"}},"Property":"zips"}}],"Values":[[{"Literal":{"Value":"\'95067\'"}}]]}}}]}]},"ExecutionMetricsKind":1}}]},"QueryId":"","ApplicationContext":{"DatasetId":"bd81ee59-dda3-44d6-a3a1-236bee18fe5d","Sources":[{"ReportId":"f129b0d3-3db0-46fd-b627-f4b0543ed028","VisualId":"b3260308f719b39b9973"}]}}],"cancelQueries":[],"modelId":369251}'

In [38]:
headers = {
    "Host": "wabi-us-gov-iowa-api.analysis.usgovcloudapi.net",
    "Connection": "keep-alive",
    "Content-Length": "2423",
    "Pragma": "no-cache",
    "Cache-Control": "no-cache",
    "Accept": "application/json, text/plain, */*",
    "RequestId": "292cbfb4-7c1f-02ee-0494-d3d613eede46",  # change
    "DNT": "1",
    "X-PowerBI-ResourceKey": "f75b3578-0ecc-401c-9e6a-b79135c32922",  # change
    "Content-Type": "application/json;charset=UTF-8",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    "ActivityId": "ae900a2e-d369-49d9-8f98-826c3af308a5",
    "Origin": "https://app.powerbigov.us",
    "Sec-Fetch-Site": "cross-site",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Dest": "empty",
    "Referer": "https://app.powerbigov.us/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9,da;q=0.8",
}
json_data = json.loads(REQUEST_BODY)

In [39]:
response = requests.post(
    POWER_BI_QUERY_URL,
    params=REQUEST_PARAMS,
    headers=headers,
    json=json_data,
    verify=False,
)

/Users/slee/.local/share/virtualenvs/california-coronavirus-scrapers-QTClc1WQ/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wabi-us-gov-iowa-api.analysis.usgovcloudapi.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [40]:
data = response.json()

In [41]:
timestamp = response.headers["Date"]

In [42]:
timestamp

'Fri, 25 Feb 2022 04:00:24 GMT'

In [43]:
ds = data["results"][0]["result"]["data"]["dsr"]["DS"][0]

In [44]:
ds

{'N': 'DS0',
 'PH': [{'DM0': [{'S': [{'N': 'G0', 'T': 1},
      {'N': 'M0', 'T': 4},
      {'N': 'H0', 'T': 4},
      {'N': 'M1', 'T': 3},
      {'N': 'H1', 'T': 3},
      {'N': 'M2', 'T': 3},
      {'N': 'H2', 'T': 3}],
     'C': ['95001', 122, 0.26, 0],
     'Ø': 84},
    {'C': ['95003', 2953, 6.41, 8.586215], 'R': 84},
    {'C': ['95005', 696, 1.51, 2.6118322], 'R': 84},
    {'C': ['95006', 820, 1.78, 3.2837968], 'R': 84},
    {'C': ['95007', 100, 0.22, 0.1275062], 'R': 84},
    {'C': ['95010', 1458, 3.16, 3.0192384], 'R': 84},
    {'C': ['95017', 77, 0.17, 0.2321704], 'R': 84},
    {'C': ['95018', 861, 1.87, 2.3322731], 'R': 84},
    {'C': ['95019', 2109, 4.58, 2.5119409], 'R': 84},
    {'C': ['95033', 343, 0.74, 3.2425448], 'R': 84},
    {'C': ['95041', 90, 0.2, 0.1486436], 'R': 84},
    {'C': ['95060', 6461, 14.02, 16.2965918], 'R': 84},
    {'C': ['95062', 5596, 12.14, 12.5245893], 'R': 84},
    {'C': ['95063', 97, 0.21, 0], 'R': 84},
    {'C': ['95064', 619, 1.34, 3.5749474], '

## Parse

In [45]:
rows = ds["PH"][0]["DM0"][1:]

In [46]:
rows

[{'C': ['95003', 2953, 6.41, 8.586215], 'R': 84},
 {'C': ['95005', 696, 1.51, 2.6118322], 'R': 84},
 {'C': ['95006', 820, 1.78, 3.2837968], 'R': 84},
 {'C': ['95007', 100, 0.22, 0.1275062], 'R': 84},
 {'C': ['95010', 1458, 3.16, 3.0192384], 'R': 84},
 {'C': ['95017', 77, 0.17, 0.2321704], 'R': 84},
 {'C': ['95018', 861, 1.87, 2.3322731], 'R': 84},
 {'C': ['95019', 2109, 4.58, 2.5119409], 'R': 84},
 {'C': ['95033', 343, 0.74, 3.2425448], 'R': 84},
 {'C': ['95041', 90, 0.2, 0.1486436], 'R': 84},
 {'C': ['95060', 6461, 14.02, 16.2965918], 'R': 84},
 {'C': ['95062', 5596, 12.14, 12.5245893], 'R': 84},
 {'C': ['95063', 97, 0.21, 0], 'R': 84},
 {'C': ['95064', 619, 1.34, 3.5749474], 'R': 84},
 {'C': ['95065', 1231, 2.67, 2.8948005], 'R': 84},
 {'C': ['95066', 1846, 4, 5.2301419], 'R': 84},
 {'C': ['95067', 46, 46, 0.1, 0.1, 0, 0]},
 {'C': ['95073', 1415, 3.07, 3.7546153], 'Ø': 84},
 {'C': ['95076', 18548, 40.24, 29.6281523], 'R': 84},
 {'C': ['Under investigation', 606, 1.31, 0], 'R': 84}]

In [47]:
confirmed_cases = []

for r in rows:
    confirmed_cases.append(r["C"][0:2])

In [48]:
confirmed_cases

[['95003', 2953],
 ['95005', 696],
 ['95006', 820],
 ['95007', 100],
 ['95010', 1458],
 ['95017', 77],
 ['95018', 861],
 ['95019', 2109],
 ['95033', 343],
 ['95041', 90],
 ['95060', 6461],
 ['95062', 5596],
 ['95063', 97],
 ['95064', 619],
 ['95065', 1231],
 ['95066', 1846],
 ['95067', 46],
 ['95073', 1415],
 ['95076', 18548],
 ['Under investigation', 606]]

In [113]:
df = pd.DataFrame(confirmed_cases, columns=["zip", "confirmed_cases"])
df.insert(0, "county", "Santa Cruz")

## Combine

Combine zips that overlap

In [114]:
pairs = {"95041": "95018", "95063": "95062", "95067": "95066"}

In [115]:
for k, v in pairs.items():
    df.loc[df.zip == v, "confirmed_cases"] = (
        df.loc[df.zip == v, "confirmed_cases"].item()
        + df.loc[df.zip == k, "confirmed_cases"].item()
    )

In [116]:
df

,county,zip,confirmed_cases
0,Santa Cruz,95003,2953
1,Santa Cruz,95005,696
2,Santa Cruz,95006,820
3,Santa Cruz,95007,100
4,Santa Cruz,95010,1458
5,Santa Cruz,95017,77
6,Santa Cruz,95018,951
7,Santa Cruz,95019,2109
8,Santa Cruz,95033,343
9,Santa Cruz,95041,90


In [122]:
df = df[~df.zip.isin(pairs.keys())].copy()

In [123]:
df["county_date"] = pd.to_datetime(timestamp)

In [124]:
df["county_date"] = df["county_date"].dt.tz_convert("US/Pacific")

In [125]:
df["county_date"] = df["county_date"].dt.strftime("%Y-%m-%d")

In [126]:
df.rename(columns={"zip": "area"}, inplace=True)

In [127]:
pairs.keys()

dict_keys(['95041', '95063', '95067'])

## Vet

In [130]:
try:
    assert not len(df) > 17
except AssertionError:
    raise AssertionError("Santa Cruz' scraper has extra rows")

In [131]:
try:
    assert not len(df) < 17
except AssertionError:
    raise AssertionError("Santa Cruz' scraper is missing rows")

## Export

Set date

In [132]:
tz = pytz.timezone("America/Los_Angeles")

In [133]:
today = datetime.now(tz).date()

In [134]:
slug = "santa-cruz"

In [135]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [136]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [137]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [138]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [139]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)